<a href="https://colab.research.google.com/github/GergelyTuri/els_project/blob/master/notebooks/videoconversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook for converting `.ffii` videos to mp4.
- 7/11/2023 GT modified indentations.

In [ ]:
! git clone https://git.ffmpeg.org/ffmpeg.git ffmpeg

fatal: destination path 'ffmpeg' already exists and is not an empty directory.


In [ ]:
import sys
import struct
import subprocess
import os
import time

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
folderpath = '/content/drive/Shareddrives/Turi_lab/Data/'\
        'PTSD_project/ELS2_PTSD/shockboxes/SEFLb/males/'

exten = '.ffii'

nbFiles = 0

In [ ]:
start_time = time.time()

for root, directories, filenames in os.walk(folderpath):
  for filename in filenames:
    if filename.lower().endswith(exten):
      filename = os.path.join(root, filename)
      print('Converting %s...\n'%filename)

      of = filename[:-5]+'.avi'

      f = open(filename,'rb')
      m = f.read(8)
      height, width = struct.unpack(">2I", m)
      rate = "15"

      cmdstr = ('ffmpeg', '-y', '-r', rate,\
        '-f', 'rawvideo',
        '-pix_fmt', 'gray',
        '-s', str(width)+"x"+str(height),
        '-i', '-',
        of)

      p = subprocess.Popen(cmdstr, stdin=subprocess.PIPE, shell=False)

      while True:
        img = f.read(width*height)
        p.stdin.write(img)
        m = f.read(8)
        if not m:
            break
        height, width = struct.unpack(">2I", m)

      print('Saved in %s'%of)

      nbFiles += 1
      p.kill()
print('Conversion of %i files over!'%nbFiles)
print("--- %.2f seconds elapsed ---" % (time.time() - start_time))
# p.terminate()